In [1]:
import json
from glob import glob
import fnmatch
from multiprocessing import Pool
from pathlib import Path
import os, itertools
from typing import *
from tqdm.notebook import tqdm
from importlib import reload
from pickle import dumps, loads, dump, load

from stexls import *

In [2]:
%rm /tmp/linker.bin

In [3]:
root = Path.home()/'MathHub'

In [4]:
try:
    with open('/tmp/linker.bin', 'rb') as fd:
        linker = load(fd)
except Exception as e:
    print(e)
    linker = Linker(root=root, file_pattern='**/*.tex')
    print(linker.update(progress=tqdm, use_multiprocessing=True))
    with open('/tmp/linker.bin', 'w+b') as fd:
        dump(linker, fd)

[Errno 2] No such file or directory: '/tmp/linker.bin'


/home/marian/MathHub/MiKoMH/AI/source/ml/en/brains.tex:2:3: File not indexed:"/home/marian/MathHub/BaseMH/physics/source/units/en/time.tex"
/home/marian/MathHub/MiKoMH/AI/source/ml/en/brains.tex:3:3: File not indexed:"/home/marian/MathHub/BaseMH/physics/source/units/en/qranges.tex"
/home/marian/MathHub/MiKoMH/AI/source/ml/en/brains.tex:4:3: File not indexed:"/home/marian/MathHub/BaseMH/physics/source/units/en/siunits-prefixed.tex"
Undefined module: "linear-regression-intro/MODULE" not defined in "/home/marian/MathHub/MiKoMH/AI/source/ml/en/linear-regression-intro.tex"
/home/marian/MathHub/MiKoMH/GenCS/source/memory/en/bitbyte.tex:2:1: File not indexed:"/home/marian/MathHub/BaseMH/physics/source/units/en/quantities.tex"
/home/marian/MathHub/MiKoMH/CAPL/source/course/fragments/acknowledgements.tex:18:5: File not indexed:"/home/marian/MathHub/MiKoMH/CAPL/source/sTeX/en/sTeX-approach.tex"
Undefined module: "symtax-base/MODULE" not defined in "/home/marian/MathHub/MiKoMH/GenCS/source/logic/

/home/marian/MathHub/MiKoMH/GenCS/source/legal/en/gdpr.tex:4:1: File not indexed:"/home/marian/MathHub/KwarcMH/SiSsi/source/winograd/cds/money.tex"
/home/marian/MathHub/MiKoMH/GenCS/source/legal/en/gdpr.tex:5:1: File not indexed:"/home/marian/MathHub/BaseMH/physics/source/units/en/siunits-prefixed.tex"
/home/marian/MathHub/MiKoMH/CAPL/source/course/fragments/proglang.tex:4:1: File not indexed:"/home/marian/MathHub/MiKoMH/CAPL/source/machines/en/microML.tex"
/home/marian/MathHub/MiKoMH/CAPL/source/course/fragments/proglang.tex:5:1: File not indexed:"/home/marian/MathHub/MiKoMH/CAPL/source/machines/en/SW-intro.tex"
/home/marian/MathHub/MiKoMH/CAPL/source/course/fragments/proglang.tex:6:1: File not indexed:"/home/marian/MathHub/MiKoMH/CAPL/source/machines/en/VM-intro.tex"
/home/marian/MathHub/MiKoMH/CAPL/source/course/fragments/proglang.tex:7:1: File not indexed:"/home/marian/MathHub/MiKoMH/CAPL/source/machines/en/asm-base.tex"
/home/marian/MathHub/MiKoMH/CAPL/source/course/fragments/prog

In [5]:
%ls -ahl /tmp/linker.bin

-rw-rw-r-- 1 marian marian 69M Apr  7 17:46 /tmp/linker.bin


In [6]:
path = root / Path('MiKoMH/talks/source/sTeX/en/sTeX-approach.tex')

In [10]:
linker.view_import_graph(path)

In [8]:
linker.update(tqdm)

{}

In [9]:
for object in linker.objects[path]:
    print(linker.links[object].format())
    print()
    print('-'*80)
    print()

TypeError: 'LinkError' object is not iterable